<a href="https://colab.research.google.com/github/deep-diver/gpt2-ft-pipeline/blob/main/notebooks/tfx_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages

Colab currently runs on Python 3.10, and TFX (`< 1.14`) only supports Python 3.8 and 3.9. Fortunately, From the `1.14rc0`, TFX supports Python 3.10.

This material was written before keras-core was announced, and there were some changes in KerasNLP and its supporting models (GPT2). I will modify the code to support latest KerasNLP version, but for now the version should be fixed as `0.5.2`

KerasNLP is used for fine-tuning while TensorFlow and TensorFlow text are required for serving time since the models from KerasNLP are built on top of its backend deep learning frameworks (TensorFlow in our case).

After installing `tfx`, you **MUST restart the runtime**

In [ ]:
!pip install tfx==1.14.0rc0
!pip install keras-nlp==0.5.2
!pip install tensorflow
!pip install tensorflow-text

Gradio and bingbong are the library to build a demo application. Gradio is to build backend and frontend of the web application, and `bingbong` is to manage conversation history in Alpaca style format.

In [ ]:
!pip install gradio
!pip install bingbong

# Download Alpaca Dataset

The easiest way to convert existing data into TFRecord format is to use TensorFlow Dataset. I have prepared one to convert JSON formatted Alpaca dataset into TFRecord in the project repository. The Alpaca dataset is from Alpaca-LoRA project which has cleaned up some mess and flaws of the original Alpaca dataset.


In [3]:
TARGET_ROOT_DIR = "alpaca"
TARGET_TRAIN_DIR = TARGET_ROOT_DIR + "/train"
TARGET_TEST_DIR = TARGET_ROOT_DIR + "/test"

!mkdir -p {TARGET_TRAIN_DIR}
!mkdir -p {TARGET_TEST_DIR}

## Get TFRecords directly

Since it takes some time to convert Alpaca dataset to TFRecord format, I have prepared and uploaded it on Google Drive. Download and move the files into the right directories as below.

In [4]:
%cd /content/

!gdown 1rxwpZY73i9aqbJgjglFmIRG7XJnT2E0m -O {TARGET_TRAIN_DIR}/alpaca-train.tfrecord
!gdown 1vZ6I9VW-4c44MJnZKm7LSs3IvXd4bWQQ -O {TARGET_TEST_DIR}/alpaca-val.tfrecord

/content
Downloading...
From: https://drive.google.com/uc?id=1rxwpZY73i9aqbJgjglFmIRG7XJnT2E0m
To: /content/alpaca/train/alpaca-train.tfrecord
100% 20.0M/20.0M [00:00<00:00, 50.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vZ6I9VW-4c44MJnZKm7LSs3IvXd4bWQQ
To: /content/alpaca/test/alpaca-val.tfrecord
100% 2.24M/2.24M [00:00<00:00, 12.9MB/s]


## Get TFRecords with TensorFlow Dataset

If you want to convert Alpaca dataset to TFRecord format yourself, please run the below cells.

In [ ]:
!git clone https://github.com/deep-diver/gpt2-ft-pipeline.git

Cloning into 'gpt2-ft-pipeline'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 200 (delta 91), reused 114 (delta 33), pack-reused 0
Receiving objects: 100% (200/200), 436.96 KiB | 4.85 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [ ]:
%cd /content/gpt2-ft-pipeline/alpaca
!tfds build

/content/gpt2-ft-pipeline/alpaca
INFO[build.py]: Loading dataset  from path: /content/gpt2-ft-pipeline/alpaca/alpaca_dataset_builder.py
2023-09-05 21:37:18.588462: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-05 21:37:18.642801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 21:37:19.684644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-05 21:37:21.711597: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Goo

In [ ]:
import tensorflow_datasets as tfds

builder = tfds.builder("alpaca")
builder.download_and_prepare()

### Move downloaded files

The returned object from `tfds.builder` knows where the TFRecords are stored, and we can get that information via `data_dir` property. We will move the TFRecord into the `/alpaca` directory for better visibility.

In [ ]:
!ls {builder.data_dir}

alpaca-train.tfrecord-00000-of-00001  dataset_info.json
alpaca-val.tfrecord-00000-of-00001    features.json


In [ ]:
%cd /content/

!cp {builder.data_dir}/alpaca-train.tfrecord-00000-of-00001 {TARGET_TRAIN_DIR}/alpaca-train.tfrecord
!cp {builder.data_dir}/alpaca-val.tfrecord-00000-of-00001 {TARGET_TEST_DIR}/alpaca-val.tfrecord

/content


# Import Packages


This notebook only works with the following versions of the required packages.
- tfx >= `1.14.0rc0`
- keras-nlp == `0.5.2`
- tensorflow >= `2.12.0`
- tensorflow-text >= `2.12.0`
- gradio >= `3.42.0`
- bingbong >= `0.4.2`

In [5]:
import tfx
import gradio
import tensorflow
import keras_nlp
import tensorflow_text
import pingpong

print(f"tfx version: {tfx.__version__}")
print(f"keras-nlp version: {keras_nlp.__version__}")
print(f"tensorflow version: {tensorflow.__version__}")
print(f"tensorflow-text version: {tensorflow_text.__version__}")
print(f"gradio version: {gradio.__version__}")
print(f"bingbong version: {pingpong.__version__}")

tfx version: 1.14.0-rc0
keras-nlp version: 0.5.2
tensorflow version: 2.13.0
tensorflow-text version: 2.13.0
gradio version: 3.42.0
bingbong version: 0.4.2


In [6]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import ImportExampleGen
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Pusher
from tfx.proto import example_gen_pb2

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental.latest_blessed_model_resolver import LatestBlessedModelResolver

import tensorflow_model_analysis as tfma

# Build TFX Pipeline

## Interactive context (a.k.a. notebook mode)

TensorFlow Extended provides **interactive context** that executes each component sequentially and interactively. This is a really good way to check if your pipeline works before moving it on to the real-world orchestrator such as `kubeflow` on Google Cloud Platform. There is no difference in the perspective of end user's view, however, there is no containerization under the hood when you use interactive context.

In [7]:
context = InteractiveContext()

## ExampleGen

It is ideal to handle a dataset in a uniform format, and TFRecord is that uniform format in TensorFlow Extended. `ExampleGen` lets you bring various formats of data (such as parquet, csv, tfrecord, ...) from different sources, then it converts them in TFRecord. Also, `ExampleGen` splits the whole dataset into train, eval, and test datasets with Python regex wildcards.

Previously, we have moved downloaded TFRecords into `/alpaca` folder, and we have moved each training and validation TFRecords into `/alpaca/train` and `/alpaca/eval` sub-directories.

`input_base` is the root directory where the TFRecords are stored, and we can specify in which sub-directories contain training and validation TFRecords via `input_config`.

In [8]:
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name="train", pattern="train/*.tfrecord"),
        example_gen_pb2.Input.Split(name="eval", pattern="test/*.tfrecord"),
    ]
)
example_gen = ImportExampleGen(
    input_base=TARGET_ROOT_DIR,
    input_config=input_config
)

In interactive context mode, each component could be run through `context.run` method. After running is finished, the running result is automatically displayed in the notebook cell. Also, you can programatically find the running result via each component's properties (i.e. `example_gen.`)

In [9]:
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

It is really important to understand what an Artifact is. To understand it concretely, we can print out the information of the emitted artifact of `ExampleGen` as below.

In [10]:
print(example_gen.outputs['examples']._artifacts[0].uri)
!ls {example_gen.outputs['examples']._artifacts[0].uri}

/tmp/tfx-interactive-2023-09-06T17_15_24.404629-ynmzkl7s/ImportExampleGen/examples/1
Split-eval  Split-train


In [11]:
!ls {example_gen.outputs['examples']._artifacts[0].uri}/Split-train

data_tfrecord-00000-of-00001.gz


## StatisticsGen

In [12]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [13]:
context.show(statistics_gen.outputs['statistics'])

## SchemaGen

In [14]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'])

context.run(schema_gen, enable_cache=False)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Transform

 `Transform` transforms a given dataset from `ExampleGen`. Since its behaviour varies from scenario to scenario, it is designed to run a custom function provided by a user. This applies to `Trainer` component as well. Hense, let's create `modules` directory to store files of those functions.

In [15]:
!mkdir modules

The following code cell is stored in `modules/preprocessing.py` file. In instruction following fine-tuning scenario, `Transform` component builds a string based on the three features of `instruction`, `input`, and `output` as well as adding system prompt (to guide text generation) in front. To do this, the following code uses:
- `tf.strings.join` to add special strings such as `### Instruction:`, `### Input:`, `### Response:`, and `<|endoftext|>`. Also it is used to add additional spaces (`\n\n`) between `### Instruction:` and `### Input:`, and between `### Input:` and `### Response:`
- `tf.cond` to conditionally run `get_prompt_with_input` or `get_prompt_without_input` functions. If the current data (row) does not have `input`, `### Input` should not be stitched.
-

In [16]:
%%writefile {"modules/preprocessing.py"}

import tensorflow as tf

def get_prompt(x):
    system_prompt = """Below are a series of dialogues between human and an AI assistant.
The AI tries to answer the given instruction as in response.
The AI MUST not generate any text containing `### Response` or `### Instruction`.
The AI MUST be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable.
The assistant MUST be happy to help with almost anything, and will do its best to understand exactly what is needed.
It also MUST avoid giving false or misleading information, and it caveats when it isn’t entirely sure about the right answer.
That said, the assistant is practical and really does its best, and doesn’t let caution get too much in the way of being useful.

"""

    def get_prompt_with_input():
        result = tf.strings.join(["### Instruction:\n", x['instruction']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Input:\n"])
        result = tf.strings.join([result, x['input']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Response:\n"])
        result = tf.strings.join([result, x['output']])
        result = tf.strings.join([result, "<|endoftext|>"])

        return tf.strings.join([system_prompt, result])

    def get_prompt_without_input():
        result = tf.strings.join(["### Instruction:\n", x['instruction']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Response:\n"])
        result = tf.strings.join([result, x['output']])
        result = tf.strings.join([result, "<|endoftext|>"])

        return tf.strings.join([system_prompt, result])

    result = tf.cond(
        tf.math.equal(x['input'], ''),
        get_prompt_with_input,
        get_prompt_without_input
    )

    # ignore instruction and input returning values. they are just there
    # since tf.map_fn requires the same structure as input in return
    return {
        'instruction': x['instruction'],
        'input': x['input'],
        'output': result
    }

def preprocessing_fn(inputs):
    inputs = tf.map_fn(lambda x: get_prompt(x), inputs)

    # final output will be mapped to `combine`
    return {
        'combine': inputs['output']
    }

Writing modules/preprocessing.py


In [17]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn="modules.preprocessing.preprocessing_fn")

In [18]:
context.run(transform, enable_cache=False)

RuntimeError: ignored

## Trainer

Just like `Transform` component, we should define a custom function about how `Trainer` should behave. It is OK to have a single Python file, but it isn't a good practice. Hence, I have separated the custom function of `Trainer` into five parts:
- `modules/train.py`: this defines an entry point `run_fn` function which `Trainer` is essentially going to call. You can see the whole workflow what `Trainer` is going to do and which other files handle which parts
  - retrieving dataset from `Transform` - `train_data.py`
  - instantiate a GPT2 model - `model.py`
  - fine-tuning the model on the dataset with given hyperparameters - `hyperparams.py`,
  - export fine-tuned model with custom signatures - `signatures.py`

In [ ]:
%%writefile {"modules/train.py"}

import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs

from .train_data import input_fn
from .signatures import model_exporter
from .model import get_gpt2_model
from .hyperparams import TRAIN_BATCH_SIZE
from .hyperparams import TRAIN_LENGTH
from .hyperparams import EPOCHS

from .utils import INFO

def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=True,
        batch_size=TRAIN_BATCH_SIZE,
    )

    for data in train_dataset.take(1):
        print(data.shape)
        print(data.numpy())

    print(f"Cardinality: {train_dataset.cardinality()}")
    model = get_gpt2_model(1463 * EPOCHS)
    model.fit(
        train_dataset,
        steps_per_epoch=TRAIN_LENGTH // TRAIN_BATCH_SIZE,
        epochs=EPOCHS
    )

    tf.saved_model.save(
        model,
        fn_args.serving_model_dir,
        signatures={
            "serving_default": model_exporter(model)
        },
    )

Writing modules/train.py


In [ ]:
%%writefile {"modules/train_data.py"}

from typing import List

import tensorflow as tf
import tensorflow_transform as tft
from tfx_bsl.tfxio import dataset_options
from tfx.components.trainer.fn_args_utils import DataAccessor

from .utils import INFO

def input_fn(
    file_pattern: List[str],
    data_accessor: DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    is_train: bool = False,
    batch_size: int = 32,
) -> tf.data.Dataset:
    INFO(f"Reading data from: {file_pattern}")

    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        dataset_options.TensorFlowDatasetOptions(
            batch_size=batch_size
        ),
        tf_transform_output.transformed_metadata.schema,
    ).map(
        lambda x: tf.squeeze(x['combine'])
    ).prefetch(
        tf.data.AUTOTUNE
    )

    return dataset

Writing modules/train_data.py


In [ ]:
%%writefile {"modules/model.py"}

from tensorflow import keras
import keras_nlp

def get_gpt2_model(cardinality):
    gpt2_tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_large_en")
    gpt2_preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
        "gpt2_large_en",
        sequence_length=256,
        add_end_token=True,
    )
    gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
        "gpt2_large_en", preprocessor=gpt2_preprocessor
    )

    learning_rate = keras.optimizers.schedules.PolynomialDecay(
        5e-5,
        decay_steps=cardinality,
        end_learning_rate=0.0,
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    gpt2_lm.compile(
        optimizer=keras.optimizers.AdamW(learning_rate),
        loss=loss,
        weighted_metrics=["accuracy"],
        sampler="random"
    )

    return gpt2_lm

Writing modules/model.py


In [ ]:
%%writefile {"modules/hyperparams.py"}

EPOCHS = 1

TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE =16

TRAIN_LENGTH = 46104
EVAL_LENGTH = 3933

Writing modules/hyperparams.py


In [ ]:
%%writefile {"modules/signatures.py"}

import tensorflow as tf

signature_dict = {
    "prompt": tf.TensorSpec(shape=[], dtype=tf.string, name="prompt"),
    "max_length": tf.TensorSpec(shape=[], dtype=tf.int64, name="max_length"),
}

def model_exporter(model):
  @tf.function(input_signature=[signature_dict])
  def serving_fn(inputs):
    prompt = tf.convert_to_tensor(inputs["prompt"])
    input_is_scalar = prompt.shape.rank == 0
    prompt = prompt[tf.newaxis] if input_is_scalar else prompt
    prompt = model.preprocessor.tokenizer(prompt)

    # Pad ragged to dense tensors.
    padded_shape = (1, inputs["max_length"])
    min_length = tf.reduce_min(prompt.row_lengths())
    input_mask = tf.ones_like(prompt, tf.bool).to_tensor(shape=padded_shape)
    prompt = prompt.to_tensor(shape=padded_shape)
    prompt = tf.cast(prompt, dtype="int64")

    generate_function = model.make_generate_function()
    output = generate_function({"token_ids": prompt, "padding_mask": input_mask}, min_length)

    token_ids, padding_mask = output["token_ids"], output["padding_mask"]
    padding_mask = padding_mask & (token_ids != model.preprocessor.tokenizer.end_token_id)
    token_ids = tf.ragged.boolean_mask(token_ids, padding_mask)

    token_ids = tf.cast(token_ids, dtype="int32")
    unicode_text = tf.strings.reduce_join(
        model.preprocessor.tokenizer.id_to_token_map.lookup(token_ids), axis=-1
    )
    split_unicode_text = tf.strings.unicode_split(unicode_text, "UTF-8")
    byte_text = tf.strings.reduce_join(
        model.preprocessor.tokenizer.unicode2byte.lookup(split_unicode_text), axis=-1
    )
    byte_text = tf.concat(byte_text, axis=0)
    byte_text = tf.squeeze(byte_text, 0)
    return {"result": byte_text}

  return serving_fn

Writing modules/signatures.py


In [ ]:
%%writefile {"modules/utils.py"}

import absl

def INFO(text: str):
    absl.logging.info(text)

Writing modules/utils.py


In [ ]:
trainer = Trainer(
    run_fn="modules.train.run_fn",
    transformed_examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
)

In [ ]:
context.run(trainer, enable_cache=False)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


(16,)
[b'Below are a series of dialogues between human and an AI assistant.\nThe AI tries to answer the given instruction as in response.\nThe AI MUST not generate any text containing `### Response` or `### Instruction`.\nThe AI MUST be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable.\nThe assistant MUST be happy to help with almost anything, and will do its best to understand exactly what is needed.\nIt also MUST avoid giving false or misleading information, and it caveats when it isn\xe2\x80\x99t entirely sure about the right answer.\nThat said, the assistant is practical and really does its best, and doesn\xe2\x80\x99t let caution get too much in the way of being useful.\n\n### Instruction:\nIdentify the main idea in the paragraph below.\n\n### Response:\nPlants produce their own food through photosynthesis.<|endoftext|>'
 b'Below are a series of dialogues between human and an AI assistant.\nThe AI tries to answer the given instruction as in r

2881/2881 [==============================] - 1317s 398ms/step - loss: 0.4373 - accuracy: 0.8841


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Pusher

In [ ]:
from tfx.components import Pusher

pusher_args = {
    "model": trainer.outputs["model"],
    "push_destination": tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory="./gpt2-serving"
        )
    ),
}
pusher = Pusher(**pusher_args)

In [ ]:
context.run(pusher, enable_cache=False)

ExecutionResult(
    component_id: Pusher
    execution_id: 6
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Build Gradio Demo

In [ ]:
saved_model_dir = pusher.outputs['pushed_model']._artifacts[0].uri

In [ ]:
!gdown 1ndXnIUDb9tcRfL7XzWMfr5-rsgzKjjNs -O saved_model.tar.gz
!tar -xvf saved_model.tar.gz

saved_model_dir = "tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/"

Downloading...
From: https://drive.google.com/uc?id=1ndXnIUDb9tcRfL7XzWMfr5-rsgzKjjNs
To: /content/saved_model.tar.gz
100% 9.31G/9.31G [00:54<00:00, 170MB/s]
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/fingerprint.pb
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/assets/
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/variables/
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/variables/variables.index
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/variables/variables.data-00000-of-00001
tmp/tfx-interactive-2023-09-05T21_52_05.838119-0e5dcnm3/Trainer/model/5/Format-Serving/saved_model.pb


In [ ]:
from typing import Text, Any, Dict, Optional

import json
import copy

import tensorflow as tf
import tensorflow_text
from tensorflow.python.saved_model import tag_constants

import gradio as gr
from pingpong import PingPong
from pingpong.gradio import GradioAlpacaChatPPManager
from pingpong.context import CtxLastWindowStrategy

local_path = "hf_model"

model = tf.saved_model.load(
    saved_model_dir,
    tags=[tag_constants.SERVING]
)
gpt_lm_predict_fn = model.signatures["serving_default"]

STYLE = """

#internet_option_radio {
    text-align: center !important;
    padding: 0px !important;
    background-color: snow;

    @media (prefers-color-scheme: dark) {
        background-color: #0b0f19;
    }
}

#internet_option_radio > .wrap:nth-child(3) {
    display: block !important;
}

.template-txt {
    text-align: center;
    font-size: 15pt !important;
}

.message {
    margin: 0px !important;
}

.load-mode-selector:nth-child(3) {
    margin: auto !important;
    text-align: center !important;
    width: fit-content !important;
}

code {
    white-space: break-spaces !important;
}

.progress-view {
    background: transparent !important;
    border-radius: 25px !important;
}

#landing-container {
    width: 85%;
    margin: auto;
}

.landing-btn {
    font-size: 2.3vw !important;
    margin-top: 25px !important;
    border-radius: 25px !important;
    height: 120px !important;

    @media screen and (max-width: 1000px) {
        font-size: 20px !important;
    }
}

#landing-bottom {
    margin-top: 20px !important;
}

.custom-btn {
    border: none !important;
    background: none !important;
    box-shadow: none !important;
    display: block !important;
    text-align: left !important;
}
.custom-btn:hover {
    background: rgb(243 244 246) !important;
}

.custom-btn-highlight {
    border: none !important;
    background: rgb(243 244 246) !important;
    box-shadow: none !important;
    display: block !important;
    text-align: left !important;

    @media (prefers-color-scheme: dark) {
      background-color: rgba(17,24,39,255) !important;
    }
}

#prompt-txt > label > span {
    display: none !important;
}
#prompt-txt > label > textarea {
    border: transparent;
    border-radius: 20px;
}
#chatbot {
    height: 800px !important;
    overflow: auto;
    box-shadow: none !important;
    border: none !important;
}
#chatbot > .wrap {
    max-height: 780px !important;
}
#chatbot + div {
  border-radius: 35px !important;
  width: 80% !important;
  margin: auto !important;
}

#left-pane {
  background-color: transparent;
  border-radius: 15px;
  padding: 10px;

  @media (prefers-color-scheme: dark) {
    background-color: rgba(31,41,55,255) !important;
  }
}

#left-top {
  padding-left: 10px;
  padding-right: 10px;
  text-align: center;
  font-weight: bold;
  font-size: large;
}

#chat-history-accordion {
  background: transparent;
  border: 0.8px !important;
}

#right-pane {
  margin-left: 20px;
  margin-right: 20px;
  background: transparent;
  border-radius: 20px;

  @media (prefers-color-scheme: dark) {
    background-color: rgba(31,41,55,255) !important;
  }

  @media screen and (max-width: 1000px) {
      margin: 0px !important;
  }
}

#initial-popup {
  z-index: 100;
  position: absolute;
  width: 50%;
  top: 50%;
  height: 50%;
  left: 50%;
  transform: translate(-50%, -50%);
  border-radius: 35px;
  padding: 15px;
}

#initial-popup-title {
  text-align: center;
  font-size: 18px;
  font-weight: bold;
}

#initial-popup-left-pane {
  min-width: 150px !important;
}

#initial-popup-right-pane {
  text-align: right;
}

.example-btn {
  padding-top: 20px !important;
  padding-bottom: 20px !important;
  padding-left: 5px !important;
  padding-right: 5px !important;
  background: linear-gradient(to bottom right, #f7faff, #ffffff) !important;
  box-shadow: none !important;
  border-radius: 20px !important;

  @media (prefers-color-scheme: dark) {
    background: rgba(70,79,86,255) !important;
  }
}

.example-btn:hover {
  box-shadow: 0.3px 0.3px 0.3px gray !important;

  @media (prefers-color-scheme: dark) {
    background: rgba(34,37,42,255) !important;
  }
}

.example-btn:active {
  @media (prefers-color-scheme: dark) {
    background: rgba(70,79,86,255) !important;
  }
}

#example-title {
  margin-bottom: 15px;
}

#aux-btns-popup {
    z-index: 200;
    position: absolute !important;
    bottom: 75px !important;
    right: 40px !important;
}

#aux-btns-popup > div {
    flex-wrap: nowrap;
    width: fit-content;
    margin: auto;
}

.aux-btn {
    height: 30px !important;
    flex-wrap: initial !important;
    flex: none !important;
    min-width: min(100px,100%) !important;
    font-weight: unset !important;
    font-size: 10pt !important;

    background: linear-gradient(to bottom right, #f7faff, #ffffff) !important;
    box-shadow: none !important;
    border-radius: 20px !important;

    opacity: 0.5;
    border-width: 0.5px;
    border-color: grey;

    color: red !important;

    @media (prefers-color-scheme: dark) {
      opacity: 0.2 !important;
      color: black !important;
    }
}

.aux-btn:hover {
    opacity: 1.0;
    box-shadow: 0.3px 0.3px 0.3px gray !important;

    @media (prefers-color-scheme: dark) {
      opacity: 1.0 !important;
      box-shadow: 0.3px 0.3px 0.3px gray !important;
    }
}

#aux-viewer {
    position: absolute !important;
    border-style: solid !important;
    overflow: visible !important;
    border: none !important;
    box-shadow: none !important;
    z-index: 1000 !important;
    opacity: 0.0 !important;
    width: 75% !important;
    right: 1px !important;
    transition: all 0.5s;
}

#aux-viewer:hover {
  opacity: 1.0 !important;
  box-shadow: 0px 0.5px 0px 0px gray !important;
}

#aux-viewer > .label-wrap {
  justify-content: end;
}

#aux-viewer > .label-wrap > span {
  margin-right: 10px;
}

#aux-viewer-inspector {
  padding: 0px;
}

#aux-viewer-inspector > label > span {
  display: none !important;
}

#aux-viewer-inspector > label > textarea {
  box-shadow: none;
  border-color: transparent;
}

#global-context > label > span {
  display: none !important;
}

#chat-back-btn {
  background: transparent !important;
}

#chat-back-btn:hover {
  @media (prefers-color-scheme: dark) {
    background: rgb(75,85,99) !important;
  }
}

#chat-back-btn:active {
  @media (prefers-color-scheme: dark) {
    background: transparent !important;
  }
}

#col-container {
    max-width: 70%;
    height: 100%;
    margin-left: auto;
    margin-right: auto;
}


#container {
  max-width: 70%;
  margin: auto;
}

#container2 {
  max-width: 60%;
  margin: auto;
}

#container3 {
  max-width: 60%;
  margin: auto;
}

.square {
  height: 100px;

  @media (prefers-color-scheme: dark) {
    background-color: rgba(70,79,86,255) !important;
  }
}

.square:hover {
  @media (prefers-color-scheme: dark) {
    background-color: rgba(34,37,42,255) !important;
  }
}

.square:active {
  @media (prefers-color-scheme: dark) {
    background-color: rgba(70,79,86,255) !important;
  }
}

.placeholders {
  min-width: max-content !important;
}

.placeholders > button {
  border-color: transparent !important;
  background-color: transparent !important;
  box-shadow: none !important;
  cursor: default !important;
}

.center {
  text-align: center;
}

.sub-container > div {
  min-width: max-content !important;
}
"""

get_local_storage = """
function() {
  globalThis.setStorage = (key, value)=>{
    localStorage.setItem(key, JSON.stringify(value));
  }
  globalThis.getStorage = (key, value)=>{
    return JSON.parse(localStorage.getItem(key));
  }
  var local_data = getStorage('local_data');
  var history = [];
  if(local_data) {
    local_data[0].pingpongs.forEach(element =>{
      history.push([element.ping, element.pong]);
    });
  }
  else {
    local_data = [];
    for (let step = 0; step < 10; step++) {
      local_data.push({'ctx': '', 'pingpongs':[]});
    }
    setStorage('local_data', local_data);
  }
  if(history.length == 0) {
    document.querySelector("#initial-popup").classList.remove('hide');
  }

  return [history, local_data];
}
"""

update_left_btns_state = """
(v)=>{
  document.querySelector('.custom-btn-highlight').classList.add('custom-btn');
  document.querySelector('.custom-btn-highlight').classList.remove('custom-btn-highlight');
  const elements = document.querySelectorAll(".custom-btn");
  for(var i=0; i < elements.length; i++) {
    const element = elements[i];
    if(element.textContent == v) {
      console.log(v);
      element.classList.add('custom-btn-highlight');
      element.classList.remove('custom-btn');
      break;
    }
  }
}"""

channels = [
    "1st Channel",
    "2nd Channel",
    "3rd Channel",
    "4th Channel",
    "5th Channel",
    "6th Channel",
    "7th Channel",
    "8th Channel",
    "9th Channel",
    "10th Channel"
]
channel_btns = []

examples = [
    "hello world",
    "what's up?",
    "this is GradioChat"
]
ex_btns = []

def reset_chat(idx, ld):
    res = [GradioAlpacaChatPPManager.from_json(json.dumps(ppm)) for ppm in ld]
    res[idx].pingpongs = []

    return (
        "",
        [],
        str(res),
        gr.update(visible=True),
        gr.update(interactive=False),
    )

def build_prompts(ppmanager):
    dummy_ppm = copy.deepcopy(ppmanager)

    dummy_ppm.ctx = """Below are a series of dialogues between human and an AI assistant.
The AI tries to answer the given instruction as in response.
The AI MUST not generate any text containing `### Response` or `### Instruction`.
The AI MUST be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable.
The assistant MUST be happy to help with almost anything, and will do its best to understand exactly what is needed.
It also MUST avoid giving false or misleading information, and it caveats when it isn’t entirely sure about the right answer.
That said, the assistant is practical and really does its best, and doesn’t let caution get too much in the way of being useful.
"""
    return CtxLastWindowStrategy(1)(dummy_ppm)

def add_pingpong(idx, ld, ping):
    res = [GradioAlpacaChatPPManager.from_json(json.dumps(ppm)) for ppm in ld]
    ppm = res[idx]

    ppm.add_pingpong(
        PingPong(ping, "")
    )

    prompt = tf.constant(build_prompts(ppm))
    max_length = tf.constant(512, dtype="int64")

    result = gpt_lm_predict_fn(
        prompt=prompt,
        max_length=max_length,
    )['result'].numpy().decode('UTF-8')
    result = result.split("### Response:")[-1].strip()

    ppm.add_pong(result)
    print(f"res:\n{str(res)}")
    return "", ppm.build_uis(), str(res)

def channel_num(btn_title):
    choice = 0

    for idx, channel in enumerate(channels):
        if channel == btn_title:
            choice = idx

    return choice

def set_chatbot(btn, ld):
    choice = channel_num(btn)

    res = [
        GradioAlpacaChatPPManager.from_json(json.dumps(ppm_str))
        for ppm_str in ld
    ]
    empty = len(res[choice].pingpongs) == 0
    return (
        res[choice].build_uis(),
        choice,
        gr.update(visible=empty)
    )

def set_example(btn):
  return btn, gr.update(visible=False)

def set_popup_visibility(ld, example_block):
  return example_block

with gr.Blocks(css=STYLE, elem_id='container-col') as demo:
    idx = gr.State(0)
    local_data = gr.JSON({},visible=False)

    with gr.Row():
        with gr.Column(scale=1, min_width=180):
            gr.Markdown("GradioChat", elem_id="left-top")

            with gr.Column(elem_id="left-pane"):
                with gr.Accordion("Histories", elem_id="chat-history-accordion"):
                    channel_btns.append(gr.Button(channels[0], elem_classes=["custom-btn-highlight"]))

                    for channel in channels[1:]:
                        channel_btns.append(gr.Button(channel, elem_classes=["custom-btn"]))

        with gr.Column(scale=8, elem_id="right-pane"):
            with gr.Column(elem_id="initial-popup", visible=False) as example_block:
                with gr.Row(scale=1):
                    with gr.Column(elem_id="initial-popup-left-pane"):
                        gr.Markdown("GradioChat", elem_id="initial-popup-title")
                        gr.Markdown("Making the community's best AI chat models available to everyone.")
                    with gr.Column(elem_id="initial-popup-right-pane"):
                        gr.Markdown("Chat UI is now open sourced on Hugging Face Hub")
                        gr.Markdown("check out the [↗ repository](https://huggingface.co/spaces/chansung/test-multi-conv)")

                with gr.Column(scale=1):
                    gr.Markdown("Examples")
                    with gr.Row() as text_block:
                        for example in examples:
                            ex_btns.append(gr.Button(example, elem_classes=["example-btn"]))

            with gr.Column(elem_id="aux-btns-popup", visible=True):
                with gr.Row():
                    # stop = gr.Button("Stop", elem_classes=["aux-btn"])
                    # regenerate = gr.Button("Regenerate", elem_classes=["aux-btn"])
                    clean = gr.Button("Clean", elem_classes=["aux-btn"])

            chatbot = gr.Chatbot(elem_id='chatbot')
            instruction_txtbox = gr.Textbox(
                placeholder="Ask anything", label="",
                elem_id="prompt-txt"
            )

    for btn in channel_btns:
        btn.click(
            set_chatbot,
            [btn, local_data],
            [chatbot, idx, example_block]
        ).then(
            None, btn, None,
            _js=update_left_btns_state
        )

    for btn in ex_btns:
        btn.click(
            set_example,
            [btn],
            [instruction_txtbox, example_block]
        )

    instruction_txtbox.submit(
        lambda: gr.update(visible=False),
        None,
        example_block
    ).then(
        add_pingpong,
        [idx, local_data, instruction_txtbox],
        [instruction_txtbox, chatbot, local_data]
    ).then(
        None, local_data, None,
        _js="(v)=>{ setStorage('local_data',v) }"
    )

    clean.click(
        reset_chat,
        [idx, local_data],
        [instruction_txtbox, chatbot, local_data, example_block]
    ).then(
        None, local_data, None,
        _js="(v)=>{ setStorage('local_data',v) }"
    )

    demo.load(
        None,
        inputs=None,
        outputs=[chatbot, local_data],
        _js=get_local_storage,
    )

demo.launch()

<ipython-input-49-401eb657204e>:552: GradioUnusedKwargWarning: You have unused kwarg parameters in Row, please remove them: {'scale': 1}
  with gr.Row(scale=1):


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>